<a href="https://colab.research.google.com/github/Suryaanugrah2001/EDA-Analisis-Sentimen-Aplikasi-Threads/blob/main/Svm_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tqdm.notebook import tqdm


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from google.colab import files
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/threads_reviews.csv")
df.head()

,source,review_description,rating,review_date
0,Google Play,Very good app for Android phone and me,5,27-08-2023 10:31
1,Google Play,Sl👍👍👍👍,5,27-08-2023 10:28
2,Google Play,Best app,5,27-08-2023 9:47
3,Google Play,Gatiya app,1,27-08-2023 9:13
4,Google Play,Lit bruv,5,27-08-2023 9:00


In [4]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
# Sentiment analysis using VADER
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

In [6]:
result = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['review_description']
    index = row['review_date']
    result[index] = sia.polarity_scores(text)

  0%|          | 0/40435 [00:00<?, ?it/s]

In [7]:
# Gabungkan skor sentimen dengan kerangka data
vaders = pd.DataFrame(result).T
vaders = vaders.reset_index().rename(columns={'index': 'review_date'})
vaders = vaders.merge(df, how='right')

In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Bersihkan dan proses awal data teks
corpus = []
for i in range(len(vaders)):
    review = re.sub(pattern='[^a-zA-Z]', repl='', string=vaders['review_description'][i])
    review = review.lower()

    review_word = review.split()
    review_word = [word for word in review_word if not word in set(stopwords.words('english'))]

    porstem = PorterStemmer()
    review = [porstem.stem(word) for word in review_word]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
# Ubah data teks menjadi fitur numerik menggunakan CountVectorizer
vectorizer = CountVectorizer(max_features=1500)
X = vectorizer.fit_transform(corpus).toarray()

In [11]:
# membuat fungsi untuk mengklasifikasikan sentimen menjadi positif, netral, dan negatif
def sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positif'
    elif compound_score > -0.05 and compound_score < 0.05:
        return 'netral'
    else:
        return 'negatif'

vaders['sentiment'] = vaders.compound.apply(sentiment)

In [12]:
# Tentukan variabel target
y = vaders['sentiment']

# Pisahkan kumpulan data menjadi data latih dan data uji, latih 80 pengujian 20( ini bisa diganti ganti pembagian datanya untuk mempengaruhi akurasi model)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Menggunakan metode Support Vector Machine (SVM)
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

In [ ]:
# Membuat prediksi pada set tes
y_pred_svm = svm_classifier.predict(X_test)

In [ ]:
# Membuat prediksi pada set tes
y_pred_svm = svm_classifier.predict(X_test)

In [ ]:
# Plot distribusi sentimen dari waktu ke waktu
vaders['review_date'] = pd.to_datetime(vaders['review_date'], dayfirst=True)
reviewbydate = vaders.groupby('review_date')['compound'].sum()

threshold = 0
plt.figure(figsize=(11, 4))
plt.plot(reviewbydate.index, reviewbydate.values, label='Sentiment', color='peru')
plt.axhline(y=threshold, color='maroon', linestyle='solid', label=f'Threshold ({threshold})')
plt.title("Distribusi Sentimen Thread berdasarkan Tanggal dan Waktu", size=15)
plt.xlabel('')
plt.ylabel('Skor Sentimen ')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Word cloud
from wordcloud import WordCloud

word = ' '.join(vaders['review_description'])
word_cloud = WordCloud(width=800,
                       height=400,
                       max_words=1000,
                       background_color='white').generate(word)

plt.figure(figsize=(10, 5))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()